In [30]:
import pandas as pd
import numpy as np

ratings_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = ratings_cols, usecols = range(3))

movie_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep = '|', names = movie_cols, usecols = range(2), encoding = "ISO-8859-1")

ratings = pd.merge(movies, ratings)

ratings.head()


,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [233]:
user_ratings = ratings.pivot_table(index = ['user_id'], columns = ['title'], values = 'rating')
p = 796
sid_rating = user_ratings.iloc[[p]]
user1_ratings = user_ratings
user_ratings.head(10)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
6,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,4.0,...,NaN,NaN,NaN,5.0,3.0,NaN,3.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
#Nomalizing the Rows.
for i in range(user_ratings.shape[0]):
    a = user_ratings.iloc[i,:].sum(axis=0)
    b = user_ratings.iloc[i,:].notnull().sum()
    #print("userId:{} avgRating: {}".format(i+1,a/b))
    k = a/b
    for j in range(user_ratings.shape[1]):
        if user_ratings.iloc[i,j] > 0:
            user_ratings.iloc[i,j] = user_ratings.iloc[i,j] - k
                   

In [172]:
user_ratings.iloc[0,2]

-1.6051660516605164

In [185]:
#Give some rating to the Testing vector and Normalize it.

pd.options.mode.chained_assignment = None 

sid_rating.iloc[0,5]    = 3.7
sid_rating.iloc[0,33]   = 2.6
sid_rating.iloc[0,79]   = 4.5
sid_rating.iloc[0,187]  = 3.4
sid_rating.iloc[0,276]  = 2.9
sid_rating.iloc[0,339]  = 4.1
sid_rating.iloc[0,444]  = 1.0
sid_rating.iloc[0,598]  = 3.9
sid_rating.iloc[0,623]  = 2.6
sid_rating.iloc[0,719]  = 5.0
sid_rating.iloc[0,856]  = 3.8
sid_rating.iloc[0,987]  = 1.9
sid_rating.iloc[0,1056] = 2.5
sid_rating.iloc[0,1134] = 4.0
sid_rating.iloc[0,1245] = 3.9
sid_rating.iloc[0,1367] = 3.0
sid_rating.iloc[0,1468] = 4.9
sid_rating.iloc[0,1532] = 4.0
sid_rating.iloc[0,1650] = 3.3

aa = sid_rating.iloc[0,:].sum(axis=0)
bb = sid_rating.iloc[0,:].notnull().sum()

kk = aa/bb

for i in range(sid_rating.shape[1]):
    if sid_rating.iloc[0,i] > 0:
        sid_rating.iloc[0,i] = sid_rating.iloc[0,i] - kk

In [187]:
sid_rating.iloc[0,33]

-0.3999999999999999

In [256]:
#Calculate Pearson Correlation.

def takesec(num):
    return num[1]

cosine_vector = []
num = 0
l1 = 0
l2 = 0
for i in range(user_ratings.shape[0]):
    for j in range(user_ratings.shape[1]):
        if user_ratings.iloc[i,j] > 0 and sid_rating.iloc[0,j] > 0:
            num = num + (user_ratings.iloc[i][j] * sid_rating.iloc[0][j])
        if user_ratings.iloc[i][j] > 0:
            l1 = l1 + (user_ratings.iloc[i][j] * user_ratings.iloc[i][j])
        if sid_rating.iloc[0][j] > 0:
            l2 = l2 + (sid_rating.iloc[0][j] * sid_rating.iloc[0][j])
    eventual_prod = np.sqrt(l1) * np.sqrt(l2)
    eventual_div = num/eventual_prod
    cosine_vector.append([i,eventual_div])
    
cosine_vector.sort(key=takesec, reverse=True)    

In [229]:
cosine_vector[940]

[387, 0.058661887386541246]

In [231]:
#Top 20 similar users are here.

cosine_vector[:20]

[[4, 0.11220748724740039],
 [1, 0.10847046693547244],
 [2, 0.10504779461368233],
 [0, 0.10402839510373235],
 [3, 0.10136182548934937],
 [5, 0.0994427952190906],
 [7, 0.09240682851126522],
 [6, 0.09081953524615928],
 [8, 0.08998464254269345],
 [9, 0.08819967895643592],
 [10, 0.08001192286408615],
 [14, 0.07753383872673522],
 [11, 0.07615384956150548],
 [15, 0.07487833416051719],
 [16, 0.07232038961979184],
 [25, 0.07186123430958347],
 [70, 0.07185788113535563],
 [69, 0.0717133960897712],
 [68, 0.07151643447668331],
 [71, 0.0713121228673115]]

In [232]:
user_ratings.iloc[0,2]

-1.6051660516605164

In [265]:
#Let's consider the top 20 users who have rated the given movie and calculate the predicted output.

predict_movie = int(input("Enter User for Movie 1 in the range 0 to 1643 : "))
count = 0
num1 = 0
den1 = 0
for i in cosine_vector:
    if user1_ratings.iloc[i[0],predict_movie] > 0:
            count = count + 1
            num1 = num1 + i[1] * user1_ratings.iloc[i[0],predict_movie]
            den1 = den1 + i[1]
        
    if count == 20:
        break

print("Expected Rating for movieId {} is : {}".format(predict_movie,num1/den1))        

Enter User for Movie 1 in the range 0 to 1643 : 79
Expected Rating for movieId 79 is : 4.50193421169317


In [ ]:
We can observe that the 79th movie was predicted to have 4.5 rating , 
which is the rating I gave to that movie in the step In[185].

Therefore we can conclude that our method is Fairly Accurate.